# GDP by State
Recently, I saw an infographic showing the GDP by state. It came as no surprise that California, Texas and New York were the top three contributors to national GDP. The article also claimed that California contributed more on a per capita basis than most states, but it did not include any more information on per capita GDP by state.  The problem was: How to compute GDP per capita for each US state.

I found the basic data at the [Bureau of Economic Analysis](https://www.bea.gov/data/gdp/gdp-state), and the dataset (an Excel workbook with multiple worksheet) that is available for download there has GDP by state. Wikipedia has data on [population by US state](https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population). This is a webpage; so you need to use one of two pandas functions either `read_html` or `read_clipboard` to load the data into a DataFrame. See the article on [Towards Data Science](https://towardsdatascience.com/2-easy-ways-to-get-tables-from-a-website-with-pandas-b92fc835e741) for more on these two functions. The first function did not import all of the columns correctly, but the ones I needed were usable.

Finally, we like to visualize data if we can. I have included a minimal amount of code to generate a [chloropleth map](https://en.wikipedia.org/wiki/Choropleth_map).

In [200]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### GDP Data
I had to go through a few iterations of looking at the Excel file and executing `pd.read_excel` to get the options right. There are multiple header lines and merged cells as well.  These are harder to ingest, but fortunately we only want data from the left half of the worksheet. 

In [201]:
df_gdp_prelim = pd.read_excel(r"D:\VSCodeWorkSpace\2024_C\data\qgdpstate0321.xlsx",
                              sheet_name='Table 3',
                              header=[1,2,3,4],
                              skiprows=0,
                              skipfooter=5,
                              index_col=0)

In [202]:
df_gdp_prelim.sample(5)

Millions of dollars                             \
              Seasonally adjusted at annual rates                              
                                             2019                              
                                               Q1       Q2       Q3       Q4   
Rhode Island                                61627    61805    61769    62335   
Illinois                                   878173   880445   890358   893356   
South Dakota                                53712    54497    55502    56052   
Florida                                   1087641  1098679  1113170  1126510   
Wisconsin                                  345242   348471   350017   353935   

                                                  Percent of the U.S.  \
                                                   Unnamed: 9_level_1   
                  2020                                           2019   
                    Q1       Q2       Q3      Q4p                  Q1   
Rhode Island     61954    56285    61081    61578                 0.3   
Illinois        884447   807383   875671   886565                 4.2   
South Dakota     56072    50951    55243    57142                 0.3   
Florida        1121367  1026676  1111614  1123896                 5.2   
Wisconsin       348021   314027   344500   348166                 1.6   

                                                                           \
              Unnamed: 10_level_1 Unnamed: 11_level_1 Unnamed: 12_level_1   
                             2019                2019                2019   
                               Q2                  Q3                  Q4   
Rhode Island                  0.3                 0.3                 0.3   
Illinois                      4.1                 4.1                 4.1   
South Dakota                  0.3                 0.3                 0.3   
Florida                       5.2                 5.2                 5.2   
Wisconsin                     1.6                 1.6                 1.6   

                                                                           \
              Unnamed: 13_level_1 Unnamed: 14_level_1 Unnamed: 15_level_1   
                             2020                2020                2020   
                               Q1                  Q2                  Q3   
Rhode Island                  0.3                 0.3                 0.3   
Illinois                      4.1                 4.1                 4.1   
South Dakota                  0.3                 0.3                 0.3   
Florida                       5.2                 5.3                 5.3   
Wisconsin                     1.6                 1.6                 1.6   

                                   
              Unnamed: 16_level_1  
                             2020  
                              Q4p  
Rhode Island                  0.3  
Illinois                      4.1  
South Dakota                  0.3  
Florida                       5.2  
Wisconsin                     1.6

In [203]:
# Just grab the 2020 dolar values.
df_gdp = df_gdp_prelim['Millions of dollars','Seasonally adjusted at annual rates',2020].copy()

C:\Users\JiangJinrui\AppData\Local\Temp\ipykernel_72988\348834615.py:2: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [204]:
df_gdp.sample(10)
# 索引列命名为STATE
df_gdp.index.name = 'STATE'
df_gdp.reset_index(inplace=True)
df_gdp
df_gdp.to_csv(r"dataset_process\gdp.csv",index=False)

### Population Data
The function `pd.read_html` is really easy to use. Note that it returns a list of tables. There is only one table on the Wikpedia source article on [US States by Population](https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population). Therefore, we can subscript it with `[0]` straight-away. Otherwise, you might need to experiment more.  I have indicated that the state name should be the index.

In [205]:
df_pop_prelim = pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population',index_col='State')[0]

In [206]:
df_pop_prelim.head(5)

,"Rank in states & territories, 2020","Rank in states & territories, 2010","Census population, April 1, 2020 [1][2]","Census population, April 1, 2010 [1][2]","Percent change, 2010–2020 [note 1]","Absolute change, 2010-2020","Total seats in the U.S. House of Representatives, 2023–2033",Census population per electoral vote [note 2],Census population per House seat,"Percent of the total U.S. population, 2020 [note 3]"
State,,,,,,,,,,
California,1,1,39538223,37253956,6.1%,+2284267,52,732189,760350,11.80%
Texas,2,2,29145505,25145561,15.9%,+3999944,38,728638,766987,8.70%
Florida,3,4,21538187,18801310,14.6%,+2736877,28,717940,769221,6.43%
New York,4,3,20201249,19378102,4.2%,+823147,26,721473,776971,6.03%
Pennsylvania,5,6,13002700,12702379,2.4%,+300321,17,684353,764865,3.88%


Obviously, there are a bunch of missing values, and if you needed these you might consider copying the table from your browser, pasting it into a cell or a text editor, and using `pd.read_clipboard.` For this exercise we don't need those rows; so we'll forge ahead!

In [207]:
df_pop = df_pop_prelim[['Census population, April 1, 2020 [1][2]']].rename(columns={'Census population, April 1, 2020 [1][2]':'Population'})

In [208]:
df_pop
df_pop['STATE']=df_pop.index
df_pop.reset_index(drop=True,inplace=True)

df_pop.to_csv(r"dataset_process\population.csv",index=False)
df_pop.head(5)

,Population,STATE
0,39538223,California
1,29145505,Texas
2,21538187,Florida
3,20201249,New York
4,13002700,Pennsylvania


## Merging the Data
Before we can merge, we need to check on the string values in the State column/index.

In [209]:
df_pop.index

RangeIndex(start=0, stop=60, step=1)

In [210]:
df_gdp.index

RangeIndex(start=0, stop=60, step=1)

The index for the GDP contains some trailing whitespace. We need to `strip` it. Note that you cannot apply a function to an index as you can for a column. Also, the population data includes some US territories while the GDP data includes some regional subtotals and a national total which we will happily ignore.

In [211]:
df_gdp.index = [x.strip() for x in df_gdp.index]

AttributeError: 'int' object has no attribute 'strip'

In [ ]:
# Verify that we won't lose information that we want to keep.
set(df_pop.index).symmetric_difference(set(df_gdp.index))

{'American Samoa',
 'Contiguous United States',
 'Far West',
 'Fifty states + D.C.',
 'Great Lakes',
 'Guam',
 'Mideast',
 'New England',
 'Northern Mariana Islands',
 'Plains',
 'Puerto Rico',
 'Rocky Mountain',
 'Southeast',
 'Southwest',
 'The fifty states',
 'Total U.S. (including D.C. and territories)',
 'U.S. Virgin Islands',
 'United States'}

In [ ]:
# We use 'State Name' here because we will reserve 'State' for the two-letter abbreviation later.
df_combined = df_gdp.merge(df_pop,left_index=True,right_index=True).reset_index().rename(columns={'index':'State Name'})

In [ ]:
# The quarterly GDP figures are annualized; so we need to sum and divide by 4 or take the mean 
# and drop the quarterly numbers.
df_combined['Annual GDP'] = df_combined[['Q1','Q2','Q3','Q4p']].mean(axis=1)
df_combined = df_combined.drop(['Q1','Q2','Q3','Q4p'],axis=1)

In [ ]:
# State GDP is in millions; so if we multiply by 1e6 we now have state GDP in dollars.
df_combined['Per Capita GDP'] = (1e6*df_combined['Annual GDP']/df_combined.Population).round(2)

In [ ]:
df_combined['Percentage of US Population'] = df_combined.Population/df_combined.Population.sum()
df_combined['Percentage of US GDP'] = df_combined['Annual GDP']/df_combined['Annual GDP'].sum()
df_combined['Ratio of State Per Capita GDP to National Average'] = df_combined['Percentage of US GDP']/df_combined['Percentage of US Population']

In [ ]:
df_combined.sort_values('Per Capita GDP',ascending=False)
df_combined.to_csv('gdp_per_capita.csv',index=False)

Almost there!  Let's create a new column based on the ratio of state per capita GDP to national GDP per capita.  This new column is the percentage of how much the state GDP per capita is above or below the national GDP per capita. 

In [ ]:
df_combined['Relative GDP per Capita'] = ((df_combined['Ratio of State Per Capita GDP to National Average'] - 1)*100).round(2)
df_combined.to_csv('dataset_process/gdp_per_capita_relative.csv',index=False)
df_combined

,State Name,Population,Annual GDP,Per Capita GDP,Percentage of US Population,Percentage of US GDP,Ratio of State Per Capita GDP to National Average,Relative GDP per Capita
0,Connecticut,3605944,280900.00,77899.16,0.010879,0.013493,1.240277,24.03
1,Maine,1362359,66195.75,48589.06,0.004110,0.003180,0.773615,-22.64
2,Massachusetts,7029917,584038.75,83079.04,0.021210,0.028055,1.322749,32.27
3,New Hampshire,1377529,85109.00,61783.82,0.004156,0.004088,0.983696,-1.63
4,Rhode Island,1097379,60224.50,54880.31,0.003311,0.002893,0.873781,-12.62
5,Vermont,643077,32797.00,51000.11,0.001940,0.001575,0.812002,-18.80
6,Delaware,989948,75512.50,76279.26,0.002987,0.003627,1.214486,21.45
7,District of Columbia,689545,143532.50,208155.38,0.002080,0.006895,3.314162,231.42
8,Maryland,6177224,422726.50,68433.09,0.018637,0.020306,1.089563,8.96
9,New Jersey,9288994,619061.50,66644.62,0.028025,0.029737,1.061087,6.11


Then we need to join to that some state abbreviations for making a chloropleth map.  I am not going to describe the chloropleth, but I will give you two reference: the [official documentation for plotly](https://plotly.com/python/choropleth-maps/)  and a how-to [GeeksForGeeks article](https://www.geeksforgeeks.org/choropleth-maps-using-plotly-in-python/).  

Next steps would be to add annotations to the map.

In [ ]:
# Plotly needs the state abbreviations. I created this Excel file just for chloropleth maps with US states.
df_abbrev = pd.read_excel(r"D:\VSCodeWorkSpace\2024_C\data\State-Abbreviations.xlsx",
                          sheet_name='Abbreviations')
df_combined=df_combined.merge(df_abbrev,on="State Name").rename(columns={'Abbreviation':'State'})

In [ ]:
import plotly.express as px

In [ ]:
fig = px.choropleth(df_combined,
                    locations="State", 
                    locationmode="USA-states", 
                    color="Relative GDP per Capita", 
                    hover_name="State",
                    scope="usa",
                    color_continuous_scale=px.colors.diverging.BrBG,
                    title="Relative Per Capita GDP")
fig.show()

This chloropleth recreates the one that started me on this journey.  

In [ ]:
fig = px.choropleth(df_combined,
                    locations="State", 
                    locationmode="USA-states", 
                    color="Annual GDP", 
                    hover_name="State",
                    scope="usa",
                    color_continuous_scale=px.colors.sequential.Greens,
                    title="Annual GDP per State")
fig.show()